In [30]:
import pandas as pd
import json
from tqdm import tqdm
import emoji
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [23]:
with open(r'source\result.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [33]:
# Создаем функцию для преобразования даты из строки в объект datetime
def parse_date(date_str):
    return datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S')

In [34]:
messages = []
participants = []
message_ids = {}

for message in tqdm(data['messages'], desc='Обрабатываем сообщения...'):
    # Если это не сообщение от участника (например, сервисное), то пропускаем его
    if message['type'] != 'message':
        continue
    
    # Если это сообщение не является текстовым, то пропускаем его
    if 'text' not in message or not message['text']:
        continue
    
    # Получаем текст сообщения
    text = emoji.demojize(str(message['text']), language='rus')
    
    # Получаем идентификатор сообщения, на которое был дан ответ
    reply_to = message.get('reply_to', {}).get('message_id')
    
    # Если это первое сообщение в списке, то добавляем его в список messages
    if not messages:
        messages.append({
            'id': message['id'],
            'date': parse_date(message['date']),
            'from': message['from'],
            'text': text,
            'reply_to': reply_to,
            'reply_to_id': None
        })
        # Добавляем отправителя первого сообщения в список участников чата
        participants.append(message['from'])
        # Добавляем идентификатор первого сообщения в словарь message_ids
        message_ids[message['id']] = 0
        continue
    
    # Получаем данные о предыдущем сообщении в списке
    prev_message = messages[-1]
    
    # Если предыдущее сообщение было отправлено тем же отправителем и в течение 5 минут,
    # то объединяем текущее сообщение с предыдущим
    if message['from'] == prev_message['from'] and parse_date(message['date']) - prev_message['date'] <= pd.Timedelta(minutes=5):
        prev_message['text'] += '\n' + text
        prev_message['reply_to'] = reply_to
    
        # Если текущее сообщение является ответом на другое сообщение, то добавляем его идентификатор
        # в словарь message_ids
        if reply_to is not None:
            prev_message['reply_to_id'] = message_ids.get(reply_to)
            message_ids[message['id']] = prev_message['reply_to_id']
        else:
            message_ids[message['id']] = len(message) - 1
    else:
        messages.append({
            'id': message['id'],
            'date': parse_date(message['date']),
            'from': message['from'],
            'text': text,
            'reply_to': reply_to,
            'reply_to_id': None
        })
        
        # Добавляем отправителя нового сообщения в список участников чата
        participants.append(message['from'])
        # Добавляем идентификатор нового сообщения в словарь message_ids
        if reply_to is not None:
            messages[-1]['reply_to_id'] = message_ids.get(reply_to, len(messages) - 1)

Обрабатываем сообщения...: 100%|██████████| 248543/248543 [00:04<00:00, 53697.98it/s]


In [36]:
df = pd.DataFrame(messages)

In [38]:
df.head(50)

,id,date,from,text,reply_to,reply_to_id
0,-999983347,2017-12-12 15:26:14,Maxim Osminin 🩼 #Slowpoke3D ࣩࣩ,"[{'type': 'mention', 'text': '@ra9ysz'}, ' Поз...",None,None
1,-999983341,2017-12-12 15:29:08,Sergemax,Не думаю )\nЕму хватает обращений,None,None
2,-999983334,2017-12-12 15:30:18,Maxim Osminin 🩼 #Slowpoke3D ࣩࣩ,"[{'type': 'mention_name', 'text': 'Sergemax', ...",None,None
3,-999983333,2017-12-12 15:30:29,Никита Пикасо,По мере возможности буду смотреть...всем привет:),None,None
4,-999983332,2017-12-12 15:31:59,Maxim Osminin 🩼 #Slowpoke3D ࣩࣩ,👍🏻\nЧуть позже заведём бота и обозначим правил...,None,None
5,-999983319,2017-12-12 16:02:31,Maxim Osminin 🩼 #Slowpoke3D ࣩࣩ,"[{'type': 'mention_name', 'text': 'Artem', 'us...",None,None
6,-999983317,2017-12-12 16:05:55,Artem Rodin,Привет привет,None,None
7,-999983316,2017-12-12 16:06:30,Maxim Osminin 🩼 #Slowpoke3D ࣩࣩ,"Думаю с началом продаж X PRO, старая прошка ст...",None,None
8,-999983314,2017-12-12 16:11:56,Artem Rodin,"Забанили?\nС другой стороны, 3dtoday не заинте...",None,None
9,-999983312,2017-12-12 16:12:53,Maxim Osminin 🩼 #Slowpoke3D ࣩࣩ,"Нет, просто перед тем как это делать я обсудил...",None,None
